# Standard Deviation and Variance

In [ ]:
import React, { useState } from 'react';
import { Play, Download, Info } from 'lucide-react';

const StableXAIDemo = () => {
  const [isRunning, setIsRunning] = useState(false);
  const [results, setResults] = useState(null);
  const [numRuns, setNumRuns] = useState(50);
  const [threshold, setThreshold] = useState(0.1);

  const runStableXAI = async () => {
    setIsRunning(true);
    setResults(null);

    // Simulate the StableXAI algorithm execution
    await new Promise(resolve => setTimeout(resolve, 1500));

    // Generate synthetic results based on the algorithm
    const features = ['age', 'income', 'credit_score', 'employment_years', 'debt_ratio'];
    
    // Simulate feature importance with variance across runs
    const explanations = [];
    for (let i = 0; i < numRuns; i++) {
      const explanation = features.map(feature => ({
        feature,
        importance: Math.random() * 0.5 + (feature === 'credit_score' ? 0.3 : 0.1)
      }));
      explanations.push(explanation);
    }

    // Calculate variance for each feature
    const featureVariances = features.map(feature => {
      const importances = explanations.map(exp => 
        exp.find(f => f.feature === feature).importance
      );
      const mean = importances.reduce((a, b) => a + b, 0) / importances.length;
      const variance = importances.reduce((sum, val) => sum + Math.pow(val - mean, 2), 0) / importances.length;
      return { feature, variance, mean };
    });

    // Identify stable features
    const maxVar = Math.max(...featureVariances.map(f => f.variance));
    const stableFeatures = featureVariances.filter(f => f.variance < threshold * maxVar);
    const unstableFeatures = featureVariances.filter(f => f.variance >= threshold * maxVar);

    // Calculate stability score
    const avgVariance = featureVariances.reduce((sum, f) => sum + f.variance, 0) / featureVariances.length;
    const stabilityScore = (1 - (avgVariance / maxVar)) * 100;

    setResults({
      allFeatures: featureVariances.sort((a, b) => b.mean - a.mean),
      stableFeatures: stableFeatures.sort((a, b) => b.mean - a.mean),
      unstableFeatures: unstableFeatures.sort((a, b) => a.variance - b.variance),
      stabilityScore: stabilityScore.toFixed(2),
      numRuns,
      threshold,
      maxVar
    });

    setIsRunning(false);
  };

  const downloadResults = () => {
    if (!results) return;
    
    const csvContent = [
      'Feature,Mean Importance,Variance,Status',
      ...results.allFeatures.map(f => {
        const status = results.stableFeatures.find(sf => sf.feature === f.feature) ? 'Stable' : 'Unstable';
        return `${f.feature},${f.mean.toFixed(4)},${f.variance.toFixed(6)},${status}`;
      })
    ].join('\n');

    const blob = new Blob([csvContent], { type: 'text/csv' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = 'stablexai_results.csv';
    a.click();
  };

  return (
    <div className="min-h-screen bg-gradient-to-br from-blue-50 to-indigo-100 p-6">
      <div className="max-w-6xl mx-auto">
        {/* Header */}
        <div className="bg-white rounded-lg shadow-lg p-6 mb-6">
          <h1 className="text-3xl font-bold text-indigo-900 mb-2">
            StableXAI Algorithm Demo
          </h1>
          <p className="text-gray-600">
            Generating stable explanations by minimizing variance across multiple runs
          </p>
          
          {/* Mathematical Formula */}
          <div className="mt-4 p-4 bg-indigo-50 rounded-lg border border-indigo-200">
            <h3 className="text-sm font-semibold text-indigo-900 mb-2">Optimization Objective:</h3>
            <div className="text-center text-lg font-mono">
              min<sub>θ</sub> 𝔼<sub>x~D</sub> [Var(φ(f(x; θ), x))]
            </div>
            <p className="text-xs text-gray-600 mt-2 text-center">
              Minimize expected variance of explanations across dataset
            </p>
          </div>
        </div>

        {/* Controls */}
        <div className="bg-white rounded-lg shadow-lg p-6 mb-6">
          <h2 className="text-xl font-semibold text-gray-800 mb-4">Configuration</h2>
          
          <div className="grid grid-cols-1 md:grid-cols-2 gap-6 mb-6">
            <div>
              <label className="block text-sm font-medium text-gray-700 mb-2">
                Number of Runs (N): {numRuns}
              </label>
              <input
                type="range"
                min="10"
                max="100"
                value={numRuns}
                onChange={(e) => setNumRuns(parseInt(e.target.value))}
                className="w-full h-2 bg-gray-200 rounded-lg appearance-none cursor-pointer"
                disabled={isRunning}
              />
              <p className="text-xs text-gray-500 mt-1">
                More runs provide better stability estimates
              </p>
            </div>

            <div>
              <label className="block text-sm font-medium text-gray-700 mb-2">
                Variance Threshold: {threshold.toFixed(2)}
              </label>
              <input
                type="range"
                min="0.01"
                max="0.5"
                step="0.01"
                value={threshold}
                onChange={(e) => setThreshold(parseFloat(e.target.value))}
                className="w-full h-2 bg-gray-200 rounded-lg appearance-none cursor-pointer"
                disabled={isRunning}
              />
              <p className="text-xs text-gray-500 mt-1">
                Features with variance below this threshold are considered stable
              </p>
            </div>
          </div>

          <button
            onClick={runStableXAI}
            disabled={isRunning}
            className="w-full bg-indigo-600 hover:bg-indigo-700 disabled:bg-gray-400 text-white font-semibold py-3 px-6 rounded-lg flex items-center justify-center gap-2 transition-colors"
          >
            <Play size={20} />
            {isRunning ? 'Running StableXAI...' : 'Run StableXAI Algorithm'}
          </button>
        </div>

        {/* Info Box */}
        <div className="bg-blue-50 border border-blue-200 rounded-lg p-4 mb-6">
          <div className="flex items-start gap-3">
            <Info className="text-blue-600 flex-shrink-0 mt-1" size={20} />
            <div className="text-sm text-blue-900">
              <p className="font-semibold mb-1">How StableXAI Works:</p>
              <ol className="list-decimal list-inside space-y-1 text-xs">
                <li>Generate explanations N times using XAI method (e.g., SHAP)</li>
                <li>Calculate variance for each feature's importance across runs</li>
                <li>Identify features with variance below threshold as stable</li>
                <li>Construct stable explanation using only stable features</li>
              </ol>
            </div>
          </div>
        </div>

        {/* Results */}
        {results && (
          <div className="space-y-6">
            {/* Stability Score */}
            <div className="bg-white rounded-lg shadow-lg p-6">
              <div className="flex justify-between items-center mb-4">
                <h2 className="text-xl font-semibold text-gray-800">Stability Analysis</h2>
                <button
                  onClick={downloadResults}
                  className="flex items-center gap-2 px-4 py-2 bg-green-600 hover:bg-green-700 text-white rounded-lg text-sm transition-colors"
                >
                  <Download size={16} />
                  Export CSV
                </button>
              </div>

              <div className="grid grid-cols-1 md:grid-cols-3 gap-4 mb-6">
                <div className="bg-gradient-to-br from-green-50 to-green-100 p-4 rounded-lg border border-green-200">
                  <p className="text-sm text-green-700 font-medium">Stability Score</p>
                  <p className="text-3xl font-bold text-green-900">{results.stabilityScore}%</p>
                  <p className="text-xs text-green-600 mt-1">Higher is better</p>
                </div>

                <div className="bg-gradient-to-br from-blue-50 to-blue-100 p-4 rounded-lg border border-blue-200">
                  <p className="text-sm text-blue-700 font-medium">Stable Features</p>
                  <p className="text-3xl font-bold text-blue-900">{results.stableFeatures.length}</p>
                  <p className="text-xs text-blue-600 mt-1">Out of {results.allFeatures.length} total</p>
                </div>

                <div className="bg-gradient-to-br from-purple-50 to-purple-100 p-4 rounded-lg border border-purple-200">
                  <p className="text-sm text-purple-700 font-medium">Runs Completed</p>
                  <p className="text-3xl font-bold text-purple-900">{results.numRuns}</p>
                  <p className="text-xs text-purple-600 mt-1">Threshold: {results.threshold.toFixed(2)}</p>
                </div>
              </div>

              {/* Formula Display */}
              <div className="bg-gray-50 p-4 rounded-lg mb-6 border border-gray-200">
                <p className="text-sm font-medium text-gray-700 mb-2">Stability Score Formula:</p>
                <div className="text-center font-mono text-sm">
                  Stability Score = (1 - Avg Variance / Max Variance) × 100
                </div>
                <p className="text-xs text-gray-600 text-center mt-2">
                  = (1 - {(results.allFeatures.reduce((sum, f) => sum + f.variance, 0) / results.allFeatures.length).toFixed(6)} / {results.maxVar.toFixed(6)}) × 100 = {results.stabilityScore}%
                </p>
              </div>

              {/* Stable Features */}
              <div className="mb-6">
                <h3 className="text-lg font-semibold text-gray-800 mb-3 flex items-center gap-2">
                  <span className="w-3 h-3 bg-green-500 rounded-full"></span>
                  Stable Features (Low Variance)
                </h3>
                <div className="space-y-2">
                  {results.stableFeatures.map((f, idx) => (
                    <div key={idx} className="bg-green-50 border border-green-200 rounded-lg p-3">
                      <div className="flex justify-between items-center mb-2">
                        <span className="font-medium text-gray-800">{f.feature}</span>
                        <span className="text-sm text-gray-600">
                          Variance: {f.variance.toFixed(6)}
                        </span>
                      </div>
                      <div className="flex items-center gap-2">
                        <div className="flex-1 bg-green-200 rounded-full h-2">
                          <div
                            className="bg-green-600 h-2 rounded-full"
                            style={{ width: `${(f.mean / Math.max(...results.allFeatures.map(x => x.mean))) * 100}%` }}
                          ></div>
                        </div>
                        <span className="text-sm font-semibold text-green-700">
                          {(f.mean * 100).toFixed(1)}%
                        </span>
                      </div>
                    </div>
                  ))}
                </div>
              </div>

              {/* Unstable Features */}
              {results.unstableFeatures.length > 0 && (
                <div>
                  <h3 className="text-lg font-semibold text-gray-800 mb-3 flex items-center gap-2">
                    <span className="w-3 h-3 bg-orange-500 rounded-full"></span>
                    Unstable Features (High Variance)
                  </h3>
                  <div className="space-y-2">
                    {results.unstableFeatures.map((f, idx) => (
                      <div key={idx} className="bg-orange-50 border border-orange-200 rounded-lg p-3">
                        <div className="flex justify-between items-center mb-2">
                          <span className="font-medium text-gray-800">{f.feature}</span>
                          <span className="text-sm text-orange-700 font-semibold">
                            Variance: {f.variance.toFixed(6)}
                          </span>
                        </div>
                        <div className="flex items-center gap-2">
                          <div className="flex-1 bg-orange-200 rounded-full h-2">
                            <div
                              className="bg-orange-600 h-2 rounded-full"
                              style={{ width: `${(f.mean / Math.max(...results.allFeatures.map(x => x.mean))) * 100}%` }}
                            ></div>
                          </div>
                          <span className="text-sm font-semibold text-orange-700">
                            {(f.mean * 100).toFixed(1)}%
                          </span>
                        </div>
                        <p className="text-xs text-orange-600 mt-2">
                          ⚠️ This feature's importance varies significantly across runs
                        </p>
                      </div>
                    ))}
                  </div>
                </div>
              )}
            </div>

            {/* Algorithm Pseudocode */}
            <div className="bg-white rounded-lg shadow-lg p-6">
              <h3 className="text-lg font-semibold text-gray-800 mb-3">Algorithm Execution Flow</h3>
              <div className="bg-gray-900 text-green-400 p-4 rounded-lg font-mono text-sm overflow-x-auto">
                <pre>{`Algorithm: StableXAI
Input: Dataset D, Model f, XAI method φ, N = ${results.numRuns}
Output: Stable explanation φ_stable

Step 1: Initialize explanation list E = []
Step 2: For i = 1 to ${results.numRuns}:
          Generate explanation φ_i = φ(f, D)
          Append φ_i to E
Step 3: Compute Var(E) for each feature
Step 4: Select features with Var < ${results.threshold.toFixed(2)} × MaxVar
Step 5: Construct φ_stable using ${results.stableFeatures.length} stable features
Step 6: Return φ_stable with stability score ${results.stabilityScore}%`}</pre>
              </div>
            </div>
          </div>
        )}
      </div>
    </div>
  );
};

export default StableXAIDemo;


Experiment with different parameters on the normal function, and see what effect it has on the shape of the distribution. How does that new shape relate to the standard deviation and variance?